In [2]:
from pyowm import OWM
from pyowm.weatherapi25.forecast import Forecast
from pyowm.exceptions.api_response_error import NotFoundError
from pyowm.exceptions.api_call_error import APICallTimeoutError
from config import OWM_API_key as key, connection_port, user, loc_host, remo_host, password, socket_path

API_key = key
owm = OWM(API_key) # the OWM object
print(type(owm))

API_key = 'key'
owm = OWM(API_key) # the OWM object
print(type(owm))

<class 'pyowm.weatherapi25.owm25.OWM25'>
<class 'pyowm.weatherapi25.owm25.OWM25'>
